# CNP_func_gridsearch_age.ipynb

In [2]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
# Parameters
# Config
toy = False
run_all = False

path = '/home/ubuntu/fmriprep/'

#AWS cloud
input_dir = '/output/' #AWS cloud
output_dir = '/output/' 

#For Daniel's computer
# input_dir = 'data/CNP_func/' 
# output_dir = '/output/' 

n_jobs = 20 #amount of cores
cv=4

description = 'CNP_func_gridsearch_age_first'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [6]:
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-09-22-05-36CNP_func197_train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']


In [14]:
print(y_train)
print(X_train.shape)
X_train = np.reshape(X_train, (84, 19306))

[29 25 42 23 29 43 27 40 32 32 22 41 23 29 26 22 27 22 36 25 47 29 40 24
 32 28 23 43 23 25 39 49 25 31 45 25 47 41 21 35 32 48 37 22 39 21 50 23
 21 28 25 40 41 25 22 35 24 21 40 26 21 24 49 37 23 30 30 49 38 27 26 22
 27 25 22 32 48 26 37 26 21 24 45 34]
(84, 19306)
(84, 19306)


## Gridsearch 
### Models
* Linear Support vector regressor
    * C = 0.0001, 0.001, 0.1, 1.0, 10.0

### Feature selection
* ANOVA
    * 5k features
    * 100k features
    * Use all features (~1.9m)

In [ ]:
from sklearn.pipeline import Pipeline

if toy:
    X_train = X_train[:4]
    y_train = y_train[:4]


# Remove features with too low between-subject variance
# variance_threshold = 

# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# feature_selection = SelectKBest(f_regression)

# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
            
anova_svr = Pipeline([
    ('anova', SelectKBest(f_regression)),
    ('svr', LinearSVR())])

parameters = [{'anova__k': [5000,100000, 'all'],
               'svr__C': [0.01, 1, 10]}]

if toy: 
    parameters = [{'anova__k': [5000],'svr__C': [0.00001, 0.0001]}]


grid = GridSearchCV(anova_svr, cv=cv, n_jobs=n_jobs, param_grid=parameters, scoring='neg_mean_absolute_error')
grid.fit(X_train, y_train)

# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






In [16]:
print('best score: ',grid.best_score_)
print('best params: ',grid.best_params_)
print('\n\nfullresults: \n', grid.cv_results_)
df1 = pd.DataFrame(grid.cv_results_)
df1

best score:  -31.737217385471077
best params:  {'anova__k': 5000, 'svr__C': 0.0001}


fullresults: 
 {'mean_fit_time': array([0.07416397, 0.07875931]), 'std_fit_time': array([0.03190736, 0.02399158]), 'mean_score_time': array([0.014516  , 0.00965482]), 'std_score_time': array([0.00588046, 0.0037135 ]), 'param_anova__k': masked_array(data=[5000, 5000],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_svr__C': masked_array(data=[1e-05, 0.0001],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'anova__k': 5000, 'svr__C': 1e-05}, {'anova__k': 5000, 'svr__C': 0.0001}], 'split0_test_score': array([-26.94593473, -26.45934728]), 'split1_test_score': array([-32.450394  , -32.00393998]), 'split2_test_score': array([-35.93470856, -35.3470856 ]), 'split3_test_score': array([-33.46384967, -33.13849669]), 'mean_test_score': array([-32.19872174, -31.73721739]), 'std_test_score': array([3.28683953, 3.27573602]), '

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_scor

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_anova__k,param_svr__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,0.074164,0.031907,0.014516,0.005880,5000,1e-05,"{'anova__k': 5000, 'svr__C': 1e-05}",-26.945935,-32.450394,-35.934709,-33.463850,-32.198722,3.286840,2,-33.938354,-32.116905,-30.946611,-31.772831,-32.193675,1.093398
1,0.078759,0.023992,0.009655,0.003713,5000,0.0001,"{'anova__k': 5000, 'svr__C': 0.0001}",-26.459347,-32.003940,-35.347086,-33.138497,-31.737217,3.275736,1,-33.383544,-31.669048,-30.466106,-31.228311,-31.686752,1.069995


In [30]:
age_pred = grid.predict(X_train)
mean_abs_error = np.mean(np.abs(age_pred-y_train))
df1 = pd.DataFrame(grid.cv_results_)

with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('best score: '+str(grid.best_score_))
    f.write('best params: '+str(grid.best_params_))
    f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
    f.write(str(df1)+'\n')
    f.write(str(age_pred)+'\n')
    f.write(str(y_train)+'\n')
    f.write(str(mean_abs_error)+'\n')
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_scor